In [37]:
import pandas as pd, numpy as np
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, roc_curve, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import time


In [38]:
train_df = pd.read_csv("data/twitter_data_train_multiclass.csv")
test_df = pd.read_csv("data/twitter_data_test_multiclass.csv")

print(train_df.shape)
print(test_df.shape)

(9446, 236)
(1667, 236)


In [39]:
to_drop = [
    'Unnamed: 0',
    'account_type', 
    'account_type_multi',
    'id', 
    'name', 
    'screen_name',
    'profile_image_url',
    'protected',
    'verified',
    'description',
    'description_processed',
    'tweets_list',
    'tweets_list_processed',
    'url',
    'profile_use_background_image', 
    'profile_background_tile'
    ]

scaler = MinMaxScaler() 
train_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']] = scaler.fit_transform(train_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']])
test_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']] = scaler.transform(test_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']])

X_train, y_train = train_df.drop(to_drop, axis=1), train_df['account_type_multi']
X_test, y_test = test_df.drop(to_drop, axis=1), test_df['account_type_multi']

# validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15)

In [40]:
from keras.utils import to_categorical
y_train_cat = to_categorical(y_train)
y_val_cat = to_categorical(y_val)
y_test_cat  = to_categorical(y_test)

In [41]:
start_time = time.time()

model = Sequential()

model.add(Dense(100, activation='relu', input_dim=220))
model.add(Dropout(.15))
model.add(Dense(500, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(6, activation='softmax'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.fit(X_train, y_train_cat, epochs=20, validation_data=(X_val, y_val_cat))

time_taken = time.time() - start_time
print("Total time taken for the program execution", time_taken) # seconds


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 100)               22100     
                                                                 
 dropout_7 (Dropout)         (None, 100)               0         
                                                                 
 dense_29 (Dense)            (None, 500)               50500     
                                                                 
 dense_30 (Dense)            (None, 500)               250500    
                                                                 
 dense_31 (Dense)            (None, 6)                 3006      
                                                                 
Total params: 326,106
Trainable params: 326,106
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
251/251 [============================

Train

In [42]:
best_clf = model

start_time = time.time()

y_pred_train = best_clf.predict(X_train)
y_pred_train = np.argmax(y_pred_train,axis=1)

time_taken = time.time() - start_time
print("Total time taken for the program execution", time_taken) # seconds
print(classification_report(y_train, y_pred_train, digits=5))
# roc_auc_score(y_test, y_pred_test)
print("F1 weighted: ", f1_score(y_train, y_pred_train, average='weighted'))

251/251 [==============================] - 0s 940us/step
Total time taken for the program execution 0.3622751235961914
              precision    recall  f1-score   support

           0    0.96858   0.99798   0.98306      1977
           1    0.99694   0.99391   0.99542      2297
           2    0.99611   0.95880   0.97710       267
           3    1.00000   0.98951   0.99473      3145
           4    1.00000   0.97445   0.98706       274
           5    1.00000   0.95652   0.97778        69

    accuracy                        0.99103      8029
   macro avg    0.99361   0.97853   0.98586      8029
weighted avg    0.99126   0.99103   0.99106      8029

F1 weighted:  0.9910589178589551


test

In [43]:
start_time = time.time()

y_pred_test = best_clf.predict(X_test)
y_pred_test = np.argmax(y_pred_test,axis=1)

time_taken = time.time() - start_time
print("Total time taken for the program execution", time_taken) # seconds
print(classification_report(y_test, y_pred_test, digits=5))
# roc_auc_score(y_test, y_pred_test)
print("F1 weighted: ", f1_score(y_test, y_pred_test, average='weighted'))

53/53 [==============================] - 0s 877us/step
Total time taken for the program execution 0.1070547103881836
              precision    recall  f1-score   support

           0    0.95519   0.98063   0.96774       413
           1    0.98069   0.99348   0.98704       460
           2    0.98182   0.93103   0.95575        58
           3    1.00000   0.98198   0.99091       666
           4    0.96154   0.94340   0.95238        53
           5    1.00000   0.94118   0.96970        17

    accuracy                        0.98140      1667
   macro avg    0.97987   0.96195   0.97059      1667
weighted avg    0.98171   0.98140   0.98144      1667

F1 weighted:  0.9814375633291933
